In [1]:
#######################################
# Yao Li
# yaoli90@illinois.edu
# 2019.10.6
#######################################

###   Importing some packages
####   The data is in EDF format and needs a special reader

In [2]:
import numpy as np
import os
import warnings

In [3]:
#Had to use pip3 install pyedflib in Git-Bash
from pyedflib import EdfReader # https://github.com/holgern/pyedflib

### Function to read and format data
#### 1. The data is distributed in several files, each file for one patient. 
#### 2. The EEG data for each patient contains several channels corresponding to different parts of the brain. Here we are only extracting the 'Cz' channel
#### 3. The data is a time-series data i.e, signal over a certain length of time. Every patient's data is not of the same time length (Eg: Patient 1 can have a recording of 3000 s, Patient 2 of 5000 s). Let the length be 'N' seconds.
#### 4. The data is sampled at a frequency of 'Fs' i.e. Fs data points in 1 second. So, in total there will be Fs(data points/sec)*N(seconds) = number of data points. 
#### 5. We need to re-format this data in this way:
<img src="DataFormatting.png">

In [4]:
def load_data():
    # A list 'file_dirs' contains all file names of eeg data.
    file_dirs = []
    for file_idx in range(52, 80):
        file_dirs.append('dataBig/eeg/eeg'+str(file_idx)+'.edf')

    data = []
    
    #for file_dirs[0]:
    for file_dir in file_dirs:
        with EdfReader(file_dir) as f:
            # list all signals (eeg channels)
            channels = f.getSignalLabels()
            # find the index of Cz channel
            for i in range(f.signals_in_file):
                if 'Cz' in channels[i]:
                    Cz_idx = i
            # grab the Cz channel signal
            Cz = f.readSignal(Cz_idx)
            # grab the sampling frequency
            Fs = int(f.getSampleFrequency(Cz_idx)) 
            # calculate the time length of the data
            N = int(np.floor(len(Cz)/Fs))
            #print(file_dir,'length: {} sec.'.format(N),'sampling frequency: {} d.p./sec.'.format(Fs),'First to second record of Cz: {} '.format(Cz[0:2]))

            ###### Your Code Here (do the reformatting and append all patients' data)
            
            #For 0 to N-1 for each patient
            for i in range(N):
                #The first index number is 0+Fs*i. For i=0, start is 0 and end is 256 (exclusive, last record is 255).
                #For i=1, start is 256 and end is 512 (exclusive, last record is 511)
                start = 0 + (Fs*i)
                end = Fs + (Fs*i)
                new_row = Cz[start:end]         
                data.append(new_row)
           #print('Appended all rows for patient ' + file_dir)
    
    #Make data into a numpy array
    data = np.array(data)
    #print('Data shape: ' + str(data.shape))
    
    return data
    
#load_data()

### Function to load the expert's labels
#### An expert 'A' has marked each 1-s interval with a label of either '0' or '1'. '0' means the expert thinks it is not a seizure, '1' means they think it is a seizure. 
#### You need to flatten the labels matrix to a single column array and remove all the 'nan' values
<img src="LabelFormatting.png">

In [5]:
def load_labels():

    file = 'dataBig/label/annotations_2017_A.csv'

    labels = []
    
    data_csv = np.genfromtxt(file, delimiter=',')
    labels.append(data_csv)
    #print(file,'shape: {}'.format(data_csv.shape))
    
    ##### Your code here #####
    
    #Make labels, which is a list, into a numpy array
    labels = np.array(labels)
    #print('New label 3D array shape: ' + str(labels.shape))
    
    #Flatten the array by getting rid of one of the dimesions- the x dimension
    labels = labels[0, :, :]
    #print('2D label array shape: ' + str(labels.shape))
    
    #Flatten the array again by lining up all the patient columns in a one dimensional array: ([1,2],[3,4]) becomes [1, 3, 2, 4]
    labels = labels.flatten('F')
    #print('1D label array shape: ' + str(labels.shape))
    
    #Remove the NaNs
    labels = labels[np.logical_not(np.isnan(labels))]

    #Make newRowNum equal to the new number of records
    newRowNum = len(labels)
    #print('newRowNum after NaNs gone: ' + str(newRowNum))
    
    #Reshape labels
    labels = labels.reshape(newRowNum, 1)
    #print('Labels FINAL shape: ' + str(labels.shape))
    
    return labels

#load_labels()

### Function for balancing the data (seizure vs healthy)
#### As you can tell from the labels, most of the labels are 0 and very few are 1. This leads to an unbalanced dataset and the model may become biased.
#### Let's choose only part of the dataset so that the number of '0's and '1's are equal.
#### Make sure you when you delete some labels, you delete the corresponding rows in the EEG data as well.
<img src="Data_Labels.png">

In [6]:
#def balance_data(data,label):
def balance_data(data, labels):

    ###### Your Code Here ######

    
    #Find out the number of seizure and healthy labels- np.where output is arr of index nums, arr of values, and dtype 
    seizure_labels = np.where(labels == 1)
    healthy_labels = np.where(labels == 0)

    num_seizure = len(seizure_labels[1])
    num_healthy = len(healthy_labels[1])
    
    
    # Second step, add the 'labels' column to the 'data' as a new column (appending labels)
    data = np.append(data, labels, axis=1)

        
    # Create two subarrays - one for healthy (whose last column = 0) and one for
    # seizure (whose last column = 1)
    healthy_arr = data[healthy_labels[0]]
    seizure_arr = data[seizure_labels[0]]

        
    # Randomly delete as many rows from the healthy subarray as it takes
    # for both arrays to become equal in size

    #Shuffle array
    np.random.shuffle(healthy_arr)
        
    #Delete everything but index row del_num = 103983 to make number of healthy rows equal to seizure rows
    #healthy_arr.shape[0]-seizure_arr.shape[0] = 103983
    healthy_arr = healthy_arr[healthy_arr.shape[0]-seizure_arr.shape[0]:]

        
    # Join back the two subarrays and separate them into 'data' and 'label'
    data = np.concatenate((healthy_arr, seizure_arr), axis=0)
    
    #Separate new label array
    labels = data[:,(data.shape[1]-1)]

    #Reshape labels array
    labels = labels.reshape(data.shape[0], 1)

    #Delete last column from data
    data = data[:,0:(data.shape[1]-1)]
    
        
    return data, labels
    
#data = load_data()
#labels = load_labels()
#balance_data(data, labels)

### Loading some machine learning packages and also the EEG-specific package

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import pyeeg

### Function to extract relevant features from the EEG time-series data (this is all EEG-specific)

In [8]:
## All done, no need to write any code here
def create_features(data,labels):
    Kmax = 5
    Tau = 4
    DE = 10
    Band = np.arange(1,86)
    Fs = 173
    F = np.zeros((data.shape[0],5))
    nan_idx = []
    for i in range(data.shape[0]):
        mat = data[i][:]
        DFA                = pyeeg.dfa(mat)
        HFD                = pyeeg.hfd(mat,Kmax)
        SVD_Entropy        = pyeeg.svd_entropy(mat,Tau,DE)
        Fisher_Information = pyeeg.fisher_info(mat,Tau,DE)
        PFD                = pyeeg.pfd(mat)
        Spectral_Entropy   = pyeeg.spectral_entropy(mat, Band, Fs, Power_Ratio=None)
        F[i,:] = [DFA, HFD, SVD_Entropy, Fisher_Information, PFD]#, Spectral_Entropy]
        if np.any(np.isnan(F[i,:])):
            nan_idx.append(i)
    F = np.delete(F, nan_idx, axis=0)
    labels = np.delete(labels, nan_idx)
    return F, labels

### Call the functions to load data, load labels, balance data

In [9]:
### Your code here #######
data = load_data()

labels = load_labels()

data_new, labels_new = balance_data(data, labels)

features, labels = create_features(data_new, labels_new)

C:\Users\Fab Lab\Dropbox\Grad School\Classes\HT 504 Software Engineering\Assignment 4\pyeeg\fractal_dimension.py:19: RuntimeWarning: divide by zero encountered in log
  L.append(numpy.log(numpy.mean(Lk)))
C:\Users\Fab Lab\Dropbox\Grad School\Classes\HT 504 Software Engineering\Assignment 4\pyeeg\spectrum.py:62: RuntimeWarning: invalid value encountered in true_divide
  Power_Ratio = Power / sum(Power)


### Create train, test data splitting (features is the X, labels is the Y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)
# Print the size of training and test dataset
print('-------------------------------------------------')
print('Dataset:')
print('X_train:', X_train.shape, 'X_test', X_test.shape)
print('-------------------------------------------------')

-------------------------------------------------
Dataset:
X_train: (18900, 5) X_test (6301, 5)
-------------------------------------------------


### Train the model using different algorithms (K Nearest Neighbors, SVM, Decision Tree, Naive Bayes)

In [11]:
# Listed below are the four classifiers you can try one-by-one
names = ["Nearest Neighbors", "Linear SVM", "Decision Tree", "Naive Bayes"]

classifiers = [KNeighborsClassifier(5),
                SVC(gamma=3, C=1),
                DecisionTreeClassifier(max_depth=5),
                GaussianNB()]


### Evaluate the performance of each algorithm with test set (tune hyperparameters to maximize)

In [12]:
clf_score=[]
with warnings.catch_warnings():
    warnings.simplefilter("ignore") # ignore a warning message of sklearn
    for name, clf in zip(names, classifiers): # loop through classifiers
        clf.fit(X_train, y_train) # train a classifier
        score = clf.score(X_test, y_test) # evaluate the accuracy
        clf_score.append([score,name])

for score in clf_score:
    print(score)

[0.6587843199492144, 'Nearest Neighbors']
[0.6518013013807332, 'Linear SVM']
[0.6935407078241549, 'Decision Tree']
[0.6179971433105856, 'Naive Bayes']
